# Project Phase II

### Research Question(s)

A person's life expentancy is a given country can be predicted by different metrics of their country, including violent crime rates, health care, diet, economic development, genetics, and happiness level.

### Data Collection and Cleaning

Have an initial draft of your data cleaning
appendix. Document every step that takes your raw data file(s) and turns it
into the analysis-ready data set that you would submit with your final
project. Include text cells describing your data collection (downloading,
scraping, surveys, etc), and text cells describing any additional data
curation/cleaning (merging data frames, filtering, transformations of
variables, etc). Include code cells for data curation/cleaning, but not
collection. Note: You should be saving data in intermediate files at several
points through this process so that you are not starting from scratch every
time you change something. This also makes sure you have
documentation for everything that you have done for your reference, but
we do not necessarily need to see all of it in code form.

- Ryan: life expectancies, violent crime
- John: health care, diet, economic develpment
- Chase: genetics, happiness level

1. Find all data
- Each person does their assigned data set
2. Clean data sets
- Each person does their assigned data set
3. Group data sets (if needed)
- Each person does their assigned data set
4. Join data sets
- Chase

In [98]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import requests
from bs4 import BeautifulSoup

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [94]:
df_life_expectancies = pd.read_csv("data\\life-expectancies.csv")
df_life_expectancies = df_life_expectancies[["Country Name","2020"]].rename(columns={"Country Name" : "Country","2020" : "Life Expectancy"})
df_life_expectancies.head(5)

,Country,Life Expectancy
0,Aruba,76.434000
1,Africa Eastern and Southern,64.325702
2,Afghanistan,65.173000
3,Africa Western and Central,58.445953
4,Angola,61.487000


### Life Expectancy Data
#### https://data.worldbank.org/indicator/SP.DYN.CDRT.IN
I found and used this dataset after discovering that the original data we wanted to use from the UN only allowed 100k rows to be downloaded. I downloaded this data and filtered down to the two columns we were looking for, country name, and the most recent year of data. I then renamed these columns to be simplified to "Country" and "Life Expectancy".

In [120]:
df_crime = pd.read_csv("data\\violent-crime-rates.csv")
df_crime = df_crime[["country","homRate","homRateYear"]].rename(columns={"country":"Country","homRate":"Homicide Rate","homRateYear":"Year"})
df_crime.head(5)

,Country,Homicide Rate,Year
0,El Salvador,52.02,2018.0
1,United States Virgin Islands,49.28,2012.0
2,Jamaica,43.85,2018.0
3,Lesotho,43.56,2015.0
4,Honduras,38.93,2018.0


## Intentional Homicide Rate
#### https://worldpopulationreview.com/country-rankings/murder-rate-by-country
I have found and used the violent crime dataset from the world population review. I then downloaded it and read it into a pandas data frame. I think narrowed it down to our columns of interest and renamed them.

### Data Description

Have an initial draft of your data description (link in syllabus) section.
Your data description should be about your analysis-ready data.

### Data Limitations

Identify any potential problems with your dataset.

### Exploratory Data Analysis

Perform an (initial) exploratory data analysis.

1. Collinearity test
2. Multivariate regression model
3. Plots
4. Summary stats

### Questions for Reviewers

List specific questions for your peer reviewers
and project mentor to answer in giving you feedback on this phase.